### Classification approach for exercise

In [1]:
import os,re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_absolute_error,mean_squared_error
from sklearn.metrics import classification_report,confusion_matrix,log_loss
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier

pd.set_option('display.max_columns',200)
pd.set_option("display.max_rows",500)

In [2]:
dfTrain = pd.read_csv("exercise_40_train.csv")
dfTest = pd.read_csv("exercise_40_test.csv")
dfTest.head(5)


,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11,x12,x13,x14,x15,x16,x17,x18,x19,x20,x21,x22,x23,x24,x25,x26,x27,x28,x29,x30,x31,x32,x33,x34,x35,x36,x37,x38,x39,x40,x41,x42,x43,x44,x45,x46,x47,x48,x49,x50,x51,x52,x53,x54,x55,x56,x57,x58,x59,x60,x61,x62,x63,x64,x65,x66,x67,x68,x69,x70,x71,x72,x73,x74,x75,x76,x77,x78,x79,x80,x81,x82,x83,x84,x85,x86,x87,x88,x89,x90,x91,x92,x93,x94,x95,x96,x97,x98,x99,x100
0,4.747627,20.509439,Wednesday,2.299105,-1.815777,-0.752166,0.0098%,-3.240309,0.587948,-0.260721,101.113628,-0.812035,3.251085,-0.004432,-0.917763,102.193597,7.097387,75.256714,$120.216189955777,8.269754,4.794438,56.134458,-0.083641,NaN,5.740955,-3.152540,924.013304,17.697125,1.402273,NaN,no,1.461654,Florida,57.930285,4.727300,66.416594,28.450466,33.499310,5-10 miles,16.776061,99971.844703,0.492812,-0.963616,NaN,NaN,1.266416,53.020326,61.840284,NaN,1825.413159,2.517924,NaN,70.880778,NaN,1.923125,0.239009,NaN,300.62999,0,May,5.461123,5.149473,1.179229,59.346822,progressive,2.219502,17.667792,5.557066,-2.030901,0.636111,-0.000234,62.363381,4.613878,2.415655,3.632041,-0.382482,mercedes,55.938387,1.0,8.325299,-1.274085,-5.663245,2.402660,-0.061040,NaN,64.002500,0.548974,3.420875,11.553023,5.904644,NaN,12.542333,no,3.107683,0.533904,12.438759,7.298306,0,NaN,93.567120
1,1.148654,19.301465,Fri,1.862200,-0.773707,-1.461276,0.0076%,0.443209,0.522113,-1.090886,104.791999,8.805876,1.651993,NaN,-1.396065,99.356609,7.117930,79.061540,$-267.562586413086,5.668347,12.377178,NaN,0.321981,female,2.058123,-0.442525,1107.628399,15.747234,2.027073,NaN,yes,0.608259,North Carolina,55.646392,0.789132,68.881807,32.242558,-1.034680,5-10 miles,11.959804,99910.554923,0.547935,1.001799,1.315020,3.229084,5.873890,49.116516,61.279131,9.360134,1818.390072,2.269700,0.336755,69.832156,2.666487,1.224195,-1.214940,NaN,300.62999,0,July,6.520113,2.101449,-0.871509,NaN,allstate,1.289800,14.402071,7.302161,-1.553879,1.352019,-0.000234,54.104054,9.010198,0.537178,1.489493,-1.106853,ford,44.322947,1.0,6.088895,1.878944,-8.237793,3.636347,3.726452,105.488589,53.387201,-0.751229,0.295234,6.212811,4.876645,-0.848567,7.213829,yes,4.276078,NaN,10.386987,12.527094,1,yes,98.607486
2,4.986860,18.769675,Saturday,1.040845,-1.548690,2.632948,-5e-04%,-1.167885,5.739275,0.222975,102.109546,7.831517,3.055358,2.036434,1.057296,NaN,10.943217,87.567827,$-311.292903116571,3.219583,31.135956,50.048638,1.174485,male,1.609278,-0.303259,1009.911508,12.008190,0.618778,NaN,no,0.680923,NaN,53.178113,0.869321,70.249633,35.207243,48.980294,5-10 miles,14.564732,100729.380783,0.096947,-0.490053,NaN,1.333292,0.750075,48.258898,63.737244,11.564194,1815.680559,1.704048,NaN,67.090400,1.547230,NaN,1.428580,NaN,300.62999,0,January,12.190433,1.793349,-0.114922,48.121885,progressive,-1.755707,14.402071,9.903803,1.720469,0.765756,-0.000234,51.522621,11.700359,-1.867170,0.383319,-1.078648,NaN,48.854080,1.0,8.711055,-0.073306,-8.739095,NaN,2.138771,NaN,49.687134,2.641871,1.718243,NaN,4.455432,1.143388,10.483928,no,2.090868,-1.780474,11.328177,11.628247,0,yes,94.578246
3,3.709183,18.374375,Tuesday,-0.169882,-2.396549,-0.784673,-0.016%,-2.662226,1.548050,0.210141,82.653354,0.436885,1.578106,NaN,-1.287913,102.410965,6.588790,71.825782,$2229.14940030076,7.459929,1.822459,88.144007,0.909556,female,8.864059,0.641209,841.889126,20.904196,0.725017,NaN,no,0.622849,Mississippi,50.311869,0.453211,65.253390,34.432292,52.756665,5-10 miles,18.503815,101476.778846,0.888038,-0.007376,-1.126059,1.129508,-0.455920,44.525657,60.008453,12.852088,2251.680231,2.915405,5.895661,75.219207,NaN,NaN,-0.415800,NaN,300.62999,0,July,6.865209,5.083537,1.685063,46.761738,geico,-0.807993,14.402071,16.576216,0.033036,0.284538,-0.000234,54.625974,13.160347,-0.329204,2.171326,-0.109125,subaru,NaN,1.0,11.742605,-0.253294,-6.641284,4.755348,2.138771,NaN,NaN,1.811825,0.461637,18.198978,3.947223,0.693646,3.862867,no,2.643847,1.662240,10.064961,10.550014,1,NaN,100.346261
4,3.801616,20.205541,Monday,2.092652,-0.732784,-0.703101,0.0186%,0.056422,2.878167,-0.457618

### Combine data frames

In [3]:
dfCombine = pd.concat([dfTrain,dfTest])
dfCombine.iloc[40000:40001,]
# dfCombine.tail(5)

,y,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11,x12,x13,x14,x15,x16,x17,x18,x19,x20,x21,x22,x23,x24,x25,x26,x27,x28,x29,x30,x31,x32,x33,x34,x35,x36,x37,x38,x39,x40,x41,x42,x43,x44,x45,x46,x47,x48,x49,x50,x51,x52,x53,x54,x55,x56,x57,x58,x59,x60,x61,x62,x63,x64,x65,x66,x67,x68,x69,x70,x71,x72,x73,x74,x75,x76,x77,x78,x79,x80,x81,x82,x83,x84,x85,x86,x87,x88,x89,x90,x91,x92,x93,x94,x95,x96,x97,x98,x99,x100
0,NaN,4.747627,20.509439,Wednesday,2.299105,-1.815777,-0.752166,0.0098%,-3.240309,0.587948,-0.260721,101.113628,-0.812035,3.251085,-0.004432,-0.917763,102.193597,7.097387,75.256714,$120.216189955777,8.269754,4.794438,56.134458,-0.083641,NaN,5.740955,-3.15254,924.013304,17.697125,1.402273,NaN,no,1.461654,Florida,57.930285,4.7273,66.416594,28.450466,33.49931,5-10 miles,16.776061,99971.844703,0.492812,-0.963616,NaN,NaN,1.266416,53.020326,61.840284,NaN,1825.413159,2.517924,NaN,70.880778,NaN,1.923125,0.239009,NaN,300.62999,0,May,5.461123,5.149473,1.179229,59.346822,progressive,2.219502,17.667792,5.557066,-2.030901,0.636111,-0.000234,62.363381,4.613878,2.415655,3.632041,-0.382482,mercedes,55.938387,1.0,8.325299,-1.274085,-5.663245,2.40266,-0.06104,NaN,64.0025,0.548974,3.420875,11.553023,5.904644,NaN,12.542333,no,3.107683,0.533904,12.438759,7.298306,0,NaN,93.56712


In [4]:
dfCombine.reset_index(drop=True,inplace=True)
dfCombine.tail(1)

,y,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11,x12,x13,x14,x15,x16,x17,x18,x19,x20,x21,x22,x23,x24,x25,x26,x27,x28,x29,x30,x31,x32,x33,x34,x35,x36,x37,x38,x39,x40,x41,x42,x43,x44,x45,x46,x47,x48,x49,x50,x51,x52,x53,x54,x55,x56,x57,x58,x59,x60,x61,x62,x63,x64,x65,x66,x67,x68,x69,x70,x71,x72,x73,x74,x75,x76,x77,x78,x79,x80,x81,x82,x83,x84,x85,x86,x87,x88,x89,x90,x91,x92,x93,x94,x95,x96,x97,x98,x99,x100
49999,NaN,2.972268,20.485504,Sunday,1.747878,0.197357,1.798676,0.0086%,-0.318751,2.858806,0.08416,99.642148,8.497151,0.878065,-0.338883,0.350812,101.065927,7.900097,76.79105,$-508.049552925335,0.443446,-0.255657,82.889762,-2.112773,female,2.764069,NaN,1028.57299,18.45714,2.473741,-0.578496,yes,0.634841,NaN,50.916587,1.271283,63.837508,29.870539,39.909804,5-10 miles,19.938424,101720.09239,-0.875818,0.765511,NaN,0.996541,-3.489898,49.483851,65.862504,7.303529,1150.95107,1.808276,9.110502,72.228876,-0.61096,2.634673,-1.92103,NaN,300.62999,0,September,6.847741,2.181149,1.297841,50.885163,progressive,0.153248,14.402071,11.865657,-1.29376,0.434589,-0.000234,52.004365,4.865971,NaN,1.524051,0.731896,NaN,45.800511,1.0,7.780886,0.953972,-6.183223,1.565181,3.140183,95.665994,NaN,-0.127836,-1.550846,11.222902,4.651827,1.397517,8.463199,yes,3.942066,1.245289,NaN,11.718582,0,yes,98.307681


### Clean data

#### Null values

In [5]:
dfCombine.isnull().sum()

y       10000
x1          0
x2          0
x3          0
x4          0
x5       3030
x6          0
x7          0
x8          0
x9          0
x10         0
x11      6439
x12         0
x13         0
x14     12292
x15         0
x16     13965
x17         0
x18         0
x19         0
x20         0
x21         0
x22      3000
x23         0
x24      4825
x25         0
x26      3050
x27         0
x28         0
x29         0
x30     40421
x31         0
x32         0
x33      8941
x34         0
x35         0
x36         0
x37         0
x38      2967
x39         0
x40         0
x41     11907
x42     12147
x43         0
x44     42813
x45     10072
x46         0
x47         0
x48         0
x49     16077
x50         0
x51         0
x52     20262
x53         0
x54     15934
x55     22120
x56         0
x57     40541
x58         0
x59         0
x60         0
x61      9059
x62         0
x63      3007
x64      6363
x65         0
x66         0
x67      3045
x68      2984
x69         0
x70         0
x71   

### Drop columns with null values & un-needed columns

In [6]:
# dfCombine.drop(["x5","x11","x14","x16","x22","x24","x26","x30","x33","x38","x41","x42","x44","x45","x49","x52","x54",\
# "x55","x57","x61","x63","x64","x67","x68","x74","x75","x76","x77","x78","x79","x80","x85","x89","x95","x99",\
# "x83","x86","x88","x91","x92","x94","x96"],axis=1,inplace=True)
# unneeded: x39
dfCombine.drop(["x11","x14","x16","x24","x30","x33","x38","x39","x41","x42","x44","x45","x49","x52","x54",\
"x55","x57","x61","x64","x74","x75","x76","x77","x78","x79","x80","x85","x89","x95","x99",\
"x83","x86","x88","x91","x96"],axis=1,inplace=True)

In [7]:
dfCombine.isnull().sum()

y       10000
x1          0
x2          0
x3          0
x4          0
x5       3030
x6          0
x7          0
x8          0
x9          0
x10         0
x12         0
x13         0
x15         0
x17         0
x18         0
x19         0
x20         0
x21         0
x22      3000
x23         0
x25         0
x26      3050
x27         0
x28         0
x29         0
x31         0
x32         0
x34         0
x35         0
x36         0
x37         0
x40         0
x43         0
x46         0
x47         0
x48         0
x50         0
x51         0
x53         0
x56         0
x58         0
x59         0
x60         0
x62         0
x63      3007
x65         0
x66         0
x67      3045
x68      2984
x69         0
x70         0
x71         0
x72         0
x73         0
x81         0
x82         0
x84         0
x87         0
x90         0
x92      3061
x93         0
x94      2955
x97         0
x98         0
x100        0
dtype: int64

In [8]:
dfCombine[["x22","x26","x63","x67","x68","x92","x94"]].sample(25)

,x22,x26,x63,x67,x68,x92,x94
1621,51.657186,0.054297,-0.296239,14.402071,NaN,2.472980,4.396681
41422,73.152582,-0.738273,-0.893668,14.402071,5.089445,4.634528,2.161436
17624,87.446040,1.138754,0.890365,14.402071,13.447521,5.091867,3.228359
23529,20.742116,-1.891363,0.959915,14.402071,16.688628,NaN,2.776178
29732,75.751070,0.899800,-1.225252,14.402071,8.219023,2.923051,9.014874
7429,85.287820,-0.667083,-0.125356,15.947023,6.404260,7.061168,3.164822
41030,109.338933,3.781027,-0.586944,14.402071,20.774977,2.847800,3.597225
48088,66.222953,-0.135390,-0.379239,14.402071,26.921119,7.219629,2.723601
41839,79.190034,-0.217559,1.390151,17.889127,NaN,4.386116,3.152890
44594,101.843312,-1.458263,-0.685962,14.402071,10.548864,-0.922135,3.957481


### Interpolate missing values

In [9]:
dfCombine.interpolate(method="linear",inplace=True)
dfCombine.sample(25)

,y,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x12,x13,x15,x17,x18,x19,x20,x21,x22,x23,x25,x26,x27,x28,x29,x31,x32,x34,x35,x36,x37,x40,x43,x46,x47,x48,x50,x51,x53,x56,x58,x59,x60,x62,x63,x65,x66,x67,x68,x69,x70,x71,x72,x73,x81,x82,x84,x87,x90,x92,x93,x94,x97,x98,x100
30047,0.0,0.129034,20.655275,Thur,-0.649516,0.968194,2.706725,-0.0073%,0.725567,2.929578,0.676750,1.315018,-0.384069,-0.129249,9.539208,82.883905,$-323.30502524963,2.000719,4.133876,82.289224,1.764128,1.840121,-3.496827,884.181795,17.692978,-2.066971,no,-0.438438,54.306835,0.332210,68.425841,28.977933,18.631818,0.454587,1.881476,55.234534,62.837352,1235.940420,2.135259,76.312198,-1.559157,300.629990,0,August,10.099719,1.438231,allstate,4.765839,14.402071,3.756187,-2.318047,0.352899,-0.000234,52.831632,1.542249,0.169200,-8.054716,2.138771,0.553644,2.588242,3.412501,no,2.159144,11.111135,0,98.638965
36818,0.0,2.446918,19.759444,Sunday,0.368502,0.305554,-3.180717,0.0038%,-0.647443,1.397845,0.986880,3.504973,11.252804,1.368710,5.445986,82.727990,$1132.72445041725,7.839938,12.116627,92.837887,-0.123014,1.828120,1.099620,982.510732,12.349654,3.082001,yes,0.480360,62.081660,2.647859,68.546115,31.134850,13.419837,1.626014,0.127554,51.244173,61.171265,1878.085278,0.198625,72.534931,0.137524,300.629990,0,January,1.014258,0.722835,esurance,0.408693,14.402071,12.985741,1.374008,1.222413,-0.000234,43.711992,4.493988,-0.105222,-7.756158,2.138771,1.669846,3.285609,2.763848,yes,4.640149,8.968113,0,101.140767
2726,0.0,1.616962,22.102751,Wednesday,0.373347,-0.228302,-1.189965,-0.0016%,-3.800200,1.503720,1.784134,0.307330,1.245299,-0.440792,6.261890,70.963916,$2018.19721514525,7.550131,1.750106,102.524199,1.491437,6.349814,-2.446570,864.697113,23.896555,-0.538971,no,-0.339018,58.315989,-0.272543,58.809888,28.227775,15.218897,-0.012074,-1.132335,37.713253,66.525908,2844.887740,0.827281,71.365320,3.082748,300.629990,0,August,1.927211,1.173815,esurance,-0.507724,14.402071,23.727991,2.497115,0.644208,-0.000234,56.281907,7.694812,-0.595975,-9.779642,2.138771,0.421605,3.146315,3.216326,no,2.192972,9.643859,0,96.430309
4808,0.0,2.160330,18.172423,Friday,-0.119245,-1.296167,-0.375400,0.0217%,0.697760,5.860093,1.116543,6.531116,4.324292,1.830355,8.016260,72.622682,$783.318422267881,2.279258,0.572965,67.943754,0.430207,5.776306,2.789674,1014.232347,18.155007,0.958061,yes,-1.394639,48.528027,1.773151,64.391581,33.357706,16.916111,-1.616339,-1.676381,40.947120,60.900809,1701.379350,0.532098,72.514743,0.755678,300.629990,0,July,2.613029,1.536575,geico,0.178477,14.402071,27.821685,2.114599,0.724725,-0.000234,44.284092,2.199435,1.748627,-1.597788,2.138771,2.364798,4.304788,0.399535,yes,5.455930,8.328170,1,109.492419
41502,0.0,5.874814,21.234737,Sat,-0.049617,0.639657,0.405171,-0.0122%,2.858754,1.316625,-0.072887,-1.088103,4.468841,-0.947185,6.375495,88.291004,$782.356025370246,6.259149,2.499548,85.603747,0.420217,3.101558,-0.667590,976.444955,13.379016,-0.723540,no,1.773363,54.522955,1.313956,74.511239,27.100418,11.038962,0.170766,1.532349,54.696679,63.263750,2856.166068,-0.965049,72.622751,0.008911,333.418796,0,August,5.862596,-0.656819,geico,1.101930,14.402071,15.734915,1.344831,0.806172,1.507353,54.462365,3.789408,-1.937529,-5.128166,2.138771,-1.526409,1.910838,9.607558,no,3.438997,6.748504,1,96.233882
30332,0.0,3.680848,21.894537,Wednesday,1.138220,-0.920892,1.071651,0.0119%,0.185329,1.855967,2.075000,6.868532,10.115555,2.856733,7.067871,70.384713,$-89.5476145381722,3.818625,2.929183,68.227211,0.360036,5.716993,2.691168,895.323106,20.438535,1.894106,no,-1.656032,50.680473,3.456903,58.281875,29.017376,19.998614,-0.272089,-1.252804,43.123526,66.436026,812.887705,-0.132512,78.746262,1.378745,300.629990,0,April,-0.566868,1.499475,farmers,-0.370589,14.402071,9.254960,2.170845,0.285468,-0.000234,47.901976,2.758961,2.488233,-1.260449,2.138771,2.691814,5.111884,3.601428,yes,3.484224,8.981324,1,98.225239
41862,0.0,0.971791,19.276649,Wednesday,-2.763395,-0.006031,2.023790,0.0036%,-0.589687,5.403731,1.102899,-1.04588

In [10]:
dfCombine.isnull().sum()

y       0
x1      0
x2      0
x3      0
x4      0
x5      0
x6      0
x7      0
x8      0
x9      0
x10     0
x12     0
x13     0
x15     0
x17     0
x18     0
x19     0
x20     0
x21     0
x22     0
x23     0
x25     0
x26     0
x27     0
x28     0
x29     0
x31     0
x32     0
x34     0
x35     0
x36     0
x37     0
x40     0
x43     0
x46     0
x47     0
x48     0
x50     0
x51     0
x53     0
x56     0
x58     0
x59     0
x60     0
x62     0
x63     0
x65     0
x66     0
x67     0
x68     0
x69     0
x70     0
x71     0
x72     0
x73     0
x81     0
x82     0
x84     0
x87     0
x90     0
x92     0
x93     0
x94     0
x97     0
x98     0
x100    0
dtype: int64

#### x3
Clean up day names

In [11]:
dfCombine["x3"].unique()

array(['Wed', 'Friday', 'Thursday', 'Tuesday', 'Sunday', 'Saturday',
       'Sat', 'Wednesday', 'Sun', 'Tue', 'Thur', 'Monday', 'Fri', 'Mon'],
      dtype=object)

In [12]:
dfCombine["x3"] = dfCombine["x3"].replace(["Mon"],"Monday")
dfCombine["x3"] = dfCombine["x3"].replace(["Tue"],"Tuesday")
dfCombine["x3"] = dfCombine["x3"].replace(["Wed"],"Wednesday")
dfCombine["x3"] = dfCombine["x3"].replace(["Thur"],"Thursday")
dfCombine["x3"] = dfCombine["x3"].replace(["Fri"],"Friday")
dfCombine["x3"] = dfCombine["x3"].replace(["Sat"],"Saturday")
dfCombine["x3"] = dfCombine["x3"].replace(["Sun"],"Sunday")

In [13]:
dfCombine["x3"].unique()

array(['Wednesday', 'Friday', 'Thursday', 'Tuesday', 'Sunday', 'Saturday',
       'Monday'], dtype=object)

### x7
Remove percent sign

In [14]:
dfCombine['x7'] = dfCombine["x7"].str.replace("%","")

In [15]:
dfCombine.head(5)

,y,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x12,x13,x15,x17,x18,x19,x20,x21,x22,x23,x25,x26,x27,x28,x29,x31,x32,x34,x35,x36,x37,x40,x43,x46,x47,x48,x50,x51,x53,x56,x58,x59,x60,x62,x63,x65,x66,x67,x68,x69,x70,x71,x72,x73,x81,x82,x84,x87,x90,x92,x93,x94,x97,x98,x100
0,0.0,0.165254,18.060003,Wednesday,1.077380,-1.339233,-1.584341,0.0062,0.220784,1.816481,1.171788,4.644568,4.814885,-0.587361,4.278444,77.709700,$-908.650758424405,7.328029,36.601967,126.384294,1.264713,3.834082,2.352406,905.491445,14.779467,0.752524,no,-0.597288,46.667221,3.159002,69.444841,32.450076,18.129499,1.227266,0.234654,53.359113,58.251700,1602.756464,2.740134,72.167186,-1.532562,300.629990,0,August,6.600008,1.252479,farmers,1.212048,12.896733,15.263491,0.518653,0.543768,-0.912506,53.521544,7.723719,-0.471594,-3.261865,0.383351,-0.162668,2.770844,0.909479,no,5.492487,7.627730,0,104.251338
1,1.0,2.441471,18.416307,Friday,1.482586,0.920817,-0.759931,0.0064,1.192441,3.513950,1.419900,1.459868,1.443983,-1.165869,8.487132,70.808566,$-1864.9622875143,6.796408,3.789176,93.037021,0.346201,4.345028,1.651579,868.784447,25.914680,1.445294,no,1.127376,47.928569,-1.013712,61.289132,35.521824,11.389081,-0.822055,1.104631,37.728834,56.052749,3350.837875,1.995635,73.964560,0.956001,300.629990,0,April,3.864254,0.057936,allstate,0.738526,14.402071,33.940951,-0.140384,1.016703,-0.000234,46.797571,7.260365,1.691428,-4.789183,2.138771,1.252022,3.703013,9.093466,no,3.346429,10.505284,1,101.230645
2,1.0,4.427278,19.188092,Thursday,0.145652,0.366093,0.709962,-8e-04,0.952323,0.782974,-1.247022,1.098525,1.216059,0.211685,8.601193,75.922820,$-543.187402955527,3.802395,7.407996,71.022413,0.070968,4.332644,-0.375737,1077.233497,13.177479,4.174536,no,2.605279,56.674425,0.108486,69.798923,30.684074,14.372443,0.150728,-2.561083,50.236892,63.975108,3756.910196,1.772648,59.428690,4.127857,182.369349,0,September,1.370661,-0.239705,geico,-0.032980,14.402071,18.211817,-0.819798,1.010811,-0.000234,48.202036,9.336021,-2.608974,-6.387984,2.138771,0.657292,3.997183,0.305170,no,4.456565,7.810979,0,109.345215
3,0.0,3.925235,19.901257,Tuesday,1.763602,-0.251926,-0.827461,-0.0057,-0.520756,1.825586,2.223038,-1.390239,3.962961,-2.046856,6.611554,74.966925,$-182.626380634258,7.728963,8.136213,121.610846,0.700954,7.294990,-0.603983,1051.655489,17.006528,2.347355,no,1.071202,59.154933,1.319711,65.408246,34.401290,13.191173,-0.965711,-2.123048,41.857197,59.226119,1961.609788,3.155214,68.671023,0.663759,300.629990,0,September,0.013162,0.318335,geico,-0.466535,14.402071,14.629914,1.389325,0.704880,-1.510949,49.882647,5.661421,1.424316,-5.431998,2.138771,2.066346,2.933707,5.971782,no,4.100022,9.178325,1,103.021970
4,0.0,2.868802,22.202473,Sunday,3.405119,0.083162,1.381504,0.0109,-0.732739,2.151990,-0.275406,7.230125,3.877312,-1.201565,9.103015,77.977156,$967.007090837503,2.272765,24.452102,56.492317,-1.102387,6.313827,0.429187,949.904947,16.962710,0.510985,yes,2.283921,46.445617,0.022747,66.662910,31.135261,17.210430,1.012093,3.477451,47.844153,55.921988,2345.195505,3.253079,71.853326,-2.273628,149.725023,0,January,1.305936,0.418272,geico,-0.195764,14.402071,12.227512,-2.951163,1.096548,-0.000234,51.349106,9.422401,0.705003,-5.664815,2.138771,0.663113,2.922302,1.046096,yes,3.234033,11.702664,0,92.925935


In [16]:
# remove "E" notation
dfCombine["x7"] = pd.to_numeric(dfCombine["x7"])
dfCombine.sample(25)

,y,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x12,x13,x15,x17,x18,x19,x20,x21,x22,x23,x25,x26,x27,x28,x29,x31,x32,x34,x35,x36,x37,x40,x43,x46,x47,x48,x50,x51,x53,x56,x58,x59,x60,x62,x63,x65,x66,x67,x68,x69,x70,x71,x72,x73,x81,x82,x84,x87,x90,x92,x93,x94,x97,x98,x100
16059,0.0,6.746305,16.796724,Monday,3.254534,2.163801,-1.126445,-0.0093,-0.865275,2.825943,1.439095,3.065339,2.788938,-2.070749,6.318686,83.797615,$-666.095240763937,9.785466,0.484637,47.469398,-0.265460,2.760505,2.962757,963.599261,12.604461,2.201521,no,-1.960284,50.613644,1.950240,72.040322,35.556626,15.906377,1.362314,-2.515353,52.098944,54.963193,2663.101913,0.016533,69.508043,-0.854215,300.629990,0,July,8.583299,0.537422,farmers,0.374894,14.402071,19.457034,-0.615804,0.378384,-0.000234,42.322398,7.188981,-2.460503,-9.088931,2.138771,-0.034863,4.190838,1.776336,no,7.762828,9.984036,1,104.395866
40120,0.0,-0.390179,22.222942,Sunday,1.780886,-1.595036,-1.559301,-0.0174,0.064994,0.550054,-2.033243,7.740443,3.651191,2.756708,7.522107,77.879052,$-71.4341382493994,2.702382,17.300925,59.307263,-0.923395,3.462874,-0.373137,1085.462432,17.199064,-1.202740,no,-0.410175,47.856205,0.751473,70.164012,28.685447,10.522858,0.783444,1.839007,45.362501,67.607809,3338.719182,2.059411,65.437415,1.777003,300.629990,0,July,1.763170,0.104069,allstate,-0.435077,14.402071,15.785253,2.686799,1.023894,-0.000234,55.756925,23.361275,0.822598,-4.311439,2.138771,-1.702275,2.150807,6.519034,no,4.682973,9.891945,0,97.004567
33291,1.0,5.078501,22.526165,Saturday,-0.249383,0.592293,3.303969,-0.0057,1.266779,-1.325727,0.290416,-2.158435,1.170947,-1.342026,8.898152,83.395304,$597.001151367478,10.371546,9.778409,33.162498,0.402762,5.357553,-1.159835,1063.554161,12.581142,1.525389,no,-1.452179,46.203987,-0.645678,71.197025,31.765654,12.894016,0.339399,-0.804772,54.893449,52.589203,2426.524776,2.883448,75.506936,1.560527,300.629990,0,February,3.848276,-0.103630,farmers,0.721676,14.402071,35.174680,-1.342139,0.771159,-0.000234,62.946433,16.367440,-0.035714,-4.290886,2.138771,1.357085,2.548949,3.840716,no,3.213232,7.399358,1,94.180307
36618,0.0,3.593527,22.235093,Thursday,0.207407,0.097663,-0.669792,-0.0181,0.882409,2.234620,-0.427874,2.248141,2.529143,1.591858,10.724202,74.520224,$-641.68523965751,2.970698,5.523308,73.436560,-0.266146,2.487682,-0.973349,1132.530230,17.710515,-0.222621,no,1.934170,54.523141,0.277772,75.467192,28.742723,10.033319,0.373156,1.360863,43.956746,68.409280,3036.552248,0.835899,61.107897,-0.228114,300.629990,0,August,1.195545,-2.830414,farmers,-1.096700,14.402071,19.956169,-1.067143,1.761489,-0.000234,49.007414,8.349114,-0.717760,-7.545349,2.138771,2.190751,2.383928,1.172765,no,4.314973,10.764127,1,99.278186
25836,0.0,4.272407,20.927760,Tuesday,0.825258,-0.186456,0.128007,0.0009,-1.950093,0.466332,1.924200,-2.952119,2.469559,-0.069540,2.077761,80.738733,$300.618003715948,12.868084,4.848346,62.231744,1.357293,1.916056,1.901622,1017.710797,18.530527,0.231781,no,-1.525562,50.776647,1.690672,67.022516,26.472140,19.959518,0.862239,-0.686130,48.018386,51.395447,1608.633295,0.557134,74.371602,0.336147,300.629990,0,December,3.265688,-0.829255,allstate,1.038950,9.246243,6.336180,-1.617412,0.195756,-0.000234,57.445993,1.947763,1.649374,-7.127625,2.138771,-1.350837,3.522354,6.132288,no,5.199324,7.790457,1,96.987368
21375,0.0,4.356859,20.624664,Monday,2.732639,-1.652429,-0.415340,0.0192,-2.862228,2.164110,0.564046,8.249131,11.989104,2.480327,8.380141,69.297506,$1139.27652801748,5.605409,17.789180,76.070304,-1.853329,4.027014,2.544807,793.910214,19.231394,4.880211,no,1.650185,46.007368,7.010206,64.508821,36.451035,15.553564,0.515266,1.638775,48.690586,53.346579,2030.696899,3.494694,67.890911,0.726401,300.629990,0,January,4.897959,1.180473,progressive,0.402265,14.402071,8.198054,-1.582424,0.210690,-0.000234,40.979505,9.977509,-1.243375,-2.458997,2.138771,-0.947235,1.918631,2.788950,no,5.894862,9.173135,0,101.248318
41278,0.0,0.545188,17.985411,Wednesday,2.553962,-0.820836,-1.398655,0.0093,-2.805065,9.4

### x19

In [17]:
# replace dollar sign
dfCombine["x19"] = dfCombine["x19"].replace(to_replace="[\$]",value="",regex=True)
dfCombine.head(10)

,y,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x12,x13,x15,x17,x18,x19,x20,x21,x22,x23,x25,x26,x27,x28,x29,x31,x32,x34,x35,x36,x37,x40,x43,x46,x47,x48,x50,x51,x53,x56,x58,x59,x60,x62,x63,x65,x66,x67,x68,x69,x70,x71,x72,x73,x81,x82,x84,x87,x90,x92,x93,x94,x97,x98,x100
0,0.0,0.165254,18.060003,Wednesday,1.077380,-1.339233,-1.584341,0.0062,0.220784,1.816481,1.171788,4.644568,4.814885,-0.587361,4.278444,77.709700,-908.650758424405,7.328029,36.601967,126.384294,1.264713,3.834082,2.352406,905.491445,14.779467,0.752524,no,-0.597288,46.667221,3.159002,69.444841,32.450076,18.129499,1.227266,0.234654,53.359113,58.251700,1602.756464,2.740134,72.167186,-1.532562,300.629990,0,August,6.600008,1.252479,farmers,1.212048,12.896733,15.263491,0.518653,0.543768,-0.912506,53.521544,7.723719,-0.471594,-3.261865,0.383351,-0.162668,2.770844,0.909479,no,5.492487,7.627730,0,104.251338
1,1.0,2.441471,18.416307,Friday,1.482586,0.920817,-0.759931,0.0064,1.192441,3.513950,1.419900,1.459868,1.443983,-1.165869,8.487132,70.808566,-1864.9622875143,6.796408,3.789176,93.037021,0.346201,4.345028,1.651579,868.784447,25.914680,1.445294,no,1.127376,47.928569,-1.013712,61.289132,35.521824,11.389081,-0.822055,1.104631,37.728834,56.052749,3350.837875,1.995635,73.964560,0.956001,300.629990,0,April,3.864254,0.057936,allstate,0.738526,14.402071,33.940951,-0.140384,1.016703,-0.000234,46.797571,7.260365,1.691428,-4.789183,2.138771,1.252022,3.703013,9.093466,no,3.346429,10.505284,1,101.230645
2,1.0,4.427278,19.188092,Thursday,0.145652,0.366093,0.709962,-0.0008,0.952323,0.782974,-1.247022,1.098525,1.216059,0.211685,8.601193,75.922820,-543.187402955527,3.802395,7.407996,71.022413,0.070968,4.332644,-0.375737,1077.233497,13.177479,4.174536,no,2.605279,56.674425,0.108486,69.798923,30.684074,14.372443,0.150728,-2.561083,50.236892,63.975108,3756.910196,1.772648,59.428690,4.127857,182.369349,0,September,1.370661,-0.239705,geico,-0.032980,14.402071,18.211817,-0.819798,1.010811,-0.000234,48.202036,9.336021,-2.608974,-6.387984,2.138771,0.657292,3.997183,0.305170,no,4.456565,7.810979,0,109.345215
3,0.0,3.925235,19.901257,Tuesday,1.763602,-0.251926,-0.827461,-0.0057,-0.520756,1.825586,2.223038,-1.390239,3.962961,-2.046856,6.611554,74.966925,-182.626380634258,7.728963,8.136213,121.610846,0.700954,7.294990,-0.603983,1051.655489,17.006528,2.347355,no,1.071202,59.154933,1.319711,65.408246,34.401290,13.191173,-0.965711,-2.123048,41.857197,59.226119,1961.609788,3.155214,68.671023,0.663759,300.629990,0,September,0.013162,0.318335,geico,-0.466535,14.402071,14.629914,1.389325,0.704880,-1.510949,49.882647,5.661421,1.424316,-5.431998,2.138771,2.066346,2.933707,5.971782,no,4.100022,9.178325,1,103.021970
4,0.0,2.868802,22.202473,Sunday,3.405119,0.083162,1.381504,0.0109,-0.732739,2.151990,-0.275406,7.230125,3.877312,-1.201565,9.103015,77.977156,967.007090837503,2.272765,24.452102,56.492317,-1.102387,6.313827,0.429187,949.904947,16.962710,0.510985,yes,2.283921,46.445617,0.022747,66.662910,31.135261,17.210430,1.012093,3.477451,47.844153,55.921988,2345.195505,3.253079,71.853326,-2.273628,149.725023,0,January,1.305936,0.418272,geico,-0.195764,14.402071,12.227512,-2.951163,1.096548,-0.000234,51.349106,9.422401,0.705003,-5.664815,2.138771,0.663113,2.922302,1.046096,yes,3.234033,11.702664,0,92.925935
5,0.0,1.926053,19.524376,Saturday,3.702574,2.300792,-0.331106,0.0079,3.085443,0.472701,1.943171,-3.582885,3.522031,-2.799700,7.283226,87.734440,271.163475698959,15.770180,44.874132,45.729121,0.320526,2.245217,4.303686,1187.333248,8.139008,2.233107,no,0.488196,45.559596,0.359074,76.835865,37.602918,12.628248,1.452704,-1.573853,52.359238,46.900080,2959.298318,1.670786,67.591932,0.493780,300.629990,0,August,4.106257,-2.246804,progressive,-0.065794,14.402071,23.446751,-2.378431,1.393887,-0.000234,49.858508,18.739751,1.887206,-3.892529,2.138771,1.027475,1.682037,5.253010,no,4.183036,5.632881,0,97.129172
6,0.0,0.094162,18.894755,Thursday,2.027918,1.580408,1.190932,0.0174,-0.331730,2.178622,-0.454172,1.762009,2.125462,-0.907832,8.6

#### Round to 6 decimal places

In [18]:
dfCombine["x19"] = dfCombine["x19"].astype(float).round(6)
dfCombine.head(10)

,y,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x12,x13,x15,x17,x18,x19,x20,x21,x22,x23,x25,x26,x27,x28,x29,x31,x32,x34,x35,x36,x37,x40,x43,x46,x47,x48,x50,x51,x53,x56,x58,x59,x60,x62,x63,x65,x66,x67,x68,x69,x70,x71,x72,x73,x81,x82,x84,x87,x90,x92,x93,x94,x97,x98,x100
0,0.0,0.165254,18.060003,Wednesday,1.077380,-1.339233,-1.584341,0.0062,0.220784,1.816481,1.171788,4.644568,4.814885,-0.587361,4.278444,77.709700,-908.650758,7.328029,36.601967,126.384294,1.264713,3.834082,2.352406,905.491445,14.779467,0.752524,no,-0.597288,46.667221,3.159002,69.444841,32.450076,18.129499,1.227266,0.234654,53.359113,58.251700,1602.756464,2.740134,72.167186,-1.532562,300.629990,0,August,6.600008,1.252479,farmers,1.212048,12.896733,15.263491,0.518653,0.543768,-0.912506,53.521544,7.723719,-0.471594,-3.261865,0.383351,-0.162668,2.770844,0.909479,no,5.492487,7.627730,0,104.251338
1,1.0,2.441471,18.416307,Friday,1.482586,0.920817,-0.759931,0.0064,1.192441,3.513950,1.419900,1.459868,1.443983,-1.165869,8.487132,70.808566,-1864.962288,6.796408,3.789176,93.037021,0.346201,4.345028,1.651579,868.784447,25.914680,1.445294,no,1.127376,47.928569,-1.013712,61.289132,35.521824,11.389081,-0.822055,1.104631,37.728834,56.052749,3350.837875,1.995635,73.964560,0.956001,300.629990,0,April,3.864254,0.057936,allstate,0.738526,14.402071,33.940951,-0.140384,1.016703,-0.000234,46.797571,7.260365,1.691428,-4.789183,2.138771,1.252022,3.703013,9.093466,no,3.346429,10.505284,1,101.230645
2,1.0,4.427278,19.188092,Thursday,0.145652,0.366093,0.709962,-0.0008,0.952323,0.782974,-1.247022,1.098525,1.216059,0.211685,8.601193,75.922820,-543.187403,3.802395,7.407996,71.022413,0.070968,4.332644,-0.375737,1077.233497,13.177479,4.174536,no,2.605279,56.674425,0.108486,69.798923,30.684074,14.372443,0.150728,-2.561083,50.236892,63.975108,3756.910196,1.772648,59.428690,4.127857,182.369349,0,September,1.370661,-0.239705,geico,-0.032980,14.402071,18.211817,-0.819798,1.010811,-0.000234,48.202036,9.336021,-2.608974,-6.387984,2.138771,0.657292,3.997183,0.305170,no,4.456565,7.810979,0,109.345215
3,0.0,3.925235,19.901257,Tuesday,1.763602,-0.251926,-0.827461,-0.0057,-0.520756,1.825586,2.223038,-1.390239,3.962961,-2.046856,6.611554,74.966925,-182.626381,7.728963,8.136213,121.610846,0.700954,7.294990,-0.603983,1051.655489,17.006528,2.347355,no,1.071202,59.154933,1.319711,65.408246,34.401290,13.191173,-0.965711,-2.123048,41.857197,59.226119,1961.609788,3.155214,68.671023,0.663759,300.629990,0,September,0.013162,0.318335,geico,-0.466535,14.402071,14.629914,1.389325,0.704880,-1.510949,49.882647,5.661421,1.424316,-5.431998,2.138771,2.066346,2.933707,5.971782,no,4.100022,9.178325,1,103.021970
4,0.0,2.868802,22.202473,Sunday,3.405119,0.083162,1.381504,0.0109,-0.732739,2.151990,-0.275406,7.230125,3.877312,-1.201565,9.103015,77.977156,967.007091,2.272765,24.452102,56.492317,-1.102387,6.313827,0.429187,949.904947,16.962710,0.510985,yes,2.283921,46.445617,0.022747,66.662910,31.135261,17.210430,1.012093,3.477451,47.844153,55.921988,2345.195505,3.253079,71.853326,-2.273628,149.725023,0,January,1.305936,0.418272,geico,-0.195764,14.402071,12.227512,-2.951163,1.096548,-0.000234,51.349106,9.422401,0.705003,-5.664815,2.138771,0.663113,2.922302,1.046096,yes,3.234033,11.702664,0,92.925935
5,0.0,1.926053,19.524376,Saturday,3.702574,2.300792,-0.331106,0.0079,3.085443,0.472701,1.943171,-3.582885,3.522031,-2.799700,7.283226,87.734440,271.163476,15.770180,44.874132,45.729121,0.320526,2.245217,4.303686,1187.333248,8.139008,2.233107,no,0.488196,45.559596,0.359074,76.835865,37.602918,12.628248,1.452704,-1.573853,52.359238,46.900080,2959.298318,1.670786,67.591932,0.493780,300.629990,0,August,4.106257,-2.246804,progressive,-0.065794,14.402071,23.446751,-2.378431,1.393887,-0.000234,49.858508,18.739751,1.887206,-3.892529,2.138771,1.027475,1.682037,5.253010,no,4.183036,5.632881,0,97.129172
6,0.0,0.094162,18.894755,Thursday,2.027918,1.580408,1.190932,0.0174,-0.331730,2.178622,-0.454172,1.762009,2.125462,-0.907832,8.631917,71.282134,749.965010,14.2208

### x31

In [19]:
# map 0 to no and 1 to yes
dfCombine["x31"] = dfCombine["x31"].map({"no":0,"yes":1})
dfCombine["x31"].unique()

array([0, 1], dtype=int64)

### x93

In [20]:
dfCombine["x93"] = dfCombine["x93"].map({"no":0,"yes":1})
dfCombine["x93"].unique()

array([0, 1], dtype=int64)

### Encode categorical variables

#### x3

In [21]:
# days of the week
x3 = pd.get_dummies(dfCombine['x3'],prefix='cat_')
dfCombine = pd.merge(
    left=dfCombine,
    right=x3,
    left_index=True,
    right_index=True,
)

dfCombine.head(15)

,y,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x12,x13,x15,x17,x18,x19,x20,x21,x22,x23,x25,x26,x27,x28,x29,x31,x32,x34,x35,x36,x37,x40,x43,x46,x47,x48,x50,x51,x53,x56,x58,x59,x60,x62,x63,x65,x66,x67,x68,x69,x70,x71,x72,x73,x81,x82,x84,x87,x90,x92,x93,x94,x97,x98,x100,cat__Friday,cat__Monday,cat__Saturday,cat__Sunday,cat__Thursday,cat__Tuesday,cat__Wednesday
0,0.0,0.165254,18.060003,Wednesday,1.077380,-1.339233,-1.584341,0.0062,0.220784,1.816481,1.171788,4.644568,4.814885,-0.587361,4.278444,77.709700,-908.650758,7.328029,36.601967,126.384294,1.264713,3.834082,2.352406,905.491445,14.779467,0.752524,0,-0.597288,46.667221,3.159002,69.444841,32.450076,18.129499,1.227266,0.234654,53.359113,58.251700,1602.756464,2.740134,72.167186,-1.532562,300.629990,0,August,6.600008,1.252479,farmers,1.212048,12.896733,15.263491,0.518653,0.543768,-0.912506,53.521544,7.723719,-0.471594,-3.261865,0.383351,-0.162668,2.770844,0.909479,0,5.492487,7.627730,0,104.251338,0,0,0,0,0,0,1
1,1.0,2.441471,18.416307,Friday,1.482586,0.920817,-0.759931,0.0064,1.192441,3.513950,1.419900,1.459868,1.443983,-1.165869,8.487132,70.808566,-1864.962288,6.796408,3.789176,93.037021,0.346201,4.345028,1.651579,868.784447,25.914680,1.445294,0,1.127376,47.928569,-1.013712,61.289132,35.521824,11.389081,-0.822055,1.104631,37.728834,56.052749,3350.837875,1.995635,73.964560,0.956001,300.629990,0,April,3.864254,0.057936,allstate,0.738526,14.402071,33.940951,-0.140384,1.016703,-0.000234,46.797571,7.260365,1.691428,-4.789183,2.138771,1.252022,3.703013,9.093466,0,3.346429,10.505284,1,101.230645,1,0,0,0,0,0,0
2,1.0,4.427278,19.188092,Thursday,0.145652,0.366093,0.709962,-0.0008,0.952323,0.782974,-1.247022,1.098525,1.216059,0.211685,8.601193,75.922820,-543.187403,3.802395,7.407996,71.022413,0.070968,4.332644,-0.375737,1077.233497,13.177479,4.174536,0,2.605279,56.674425,0.108486,69.798923,30.684074,14.372443,0.150728,-2.561083,50.236892,63.975108,3756.910196,1.772648,59.428690,4.127857,182.369349,0,September,1.370661,-0.239705,geico,-0.032980,14.402071,18.211817,-0.819798,1.010811,-0.000234,48.202036,9.336021,-2.608974,-6.387984,2.138771,0.657292,3.997183,0.305170,0,4.456565,7.810979,0,109.345215,0,0,0,0,1,0,0
3,0.0,3.925235,19.901257,Tuesday,1.763602,-0.251926,-0.827461,-0.0057,-0.520756,1.825586,2.223038,-1.390239,3.962961,-2.046856,6.611554,74.966925,-182.626381,7.728963,8.136213,121.610846,0.700954,7.294990,-0.603983,1051.655489,17.006528,2.347355,0,1.071202,59.154933,1.319711,65.408246,34.401290,13.191173,-0.965711,-2.123048,41.857197,59.226119,1961.609788,3.155214,68.671023,0.663759,300.629990,0,September,0.013162,0.318335,geico,-0.466535,14.402071,14.629914,1.389325,0.704880,-1.510949,49.882647,5.661421,1.424316,-5.431998,2.138771,2.066346,2.933707,5.971782,0,4.100022,9.178325,1,103.021970,0,0,0,0,0,1,0
4,0.0,2.868802,22.202473,Sunday,3.405119,0.083162,1.381504,0.0109,-0.732739,2.151990,-0.275406,7.230125,3.877312,-1.201565,9.103015,77.977156,967.007091,2.272765,24.452102,56.492317,-1.102387,6.313827,0.429187,949.904947,16.962710,0.510985,1,2.283921,46.445617,0.022747,66.662910,31.135261,17.210430,1.012093,3.477451,47.844153,55.921988,2345.195505,3.253079,71.853326,-2.273628,149.725023,0,January,1.305936,0.418272,geico,-0.195764,14.402071,12.227512,-2.951163,1.096548,-0.000234,51.349106,9.422401,0.705003,-5.664815,2.138771,0.663113,2.922302,1.046096,1,3.234033,11.702664,0,92.925935,0,0,0,1,0,0,0
5,0.0,1.926053,19.524376,Saturday,3.702574,2.300792,-0.331106,0.0079,3.085443,0.472701,1.943171,-3.582885,3.522031,-2.799700,7.283226,87.734440,271.163476,15.770180,44.874132,45.729121,0.320526,2.245217,4.303686,1187.333248,8.139008,2.233107,0,0.488196,45.559596,0.359074,76.835865,37.602918,12.628248,1.452704,-1.573853,52.359238,46.900080,2959.298318,1.670786,67.591932,0.493780,300.629990,0,August,4.106257,-2.246804,progressive,-0.065794,14.402071,23.446751,-2.378431,1.393887,-0.000234,49.858508,18.739751,1.887206,-3.892529,2.138771,1.027475,1.682037,5.253010,0,4.183036,5.632881,0,97.129172,0,0,1,0,0,0,0


#### x60

In [22]:
# months
x60 = pd.get_dummies(dfCombine['x60'],prefix='cat_')
dfCombine = pd.merge(
    left=dfCombine,
    right=x60,
    left_index=True,
    right_index=True,
)

dfCombine.head(15)

,y,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x12,x13,x15,x17,x18,x19,x20,x21,x22,x23,x25,x26,x27,x28,x29,x31,x32,x34,x35,x36,x37,x40,x43,x46,x47,x48,x50,x51,x53,x56,x58,x59,x60,x62,x63,x65,x66,x67,x68,x69,x70,x71,x72,x73,x81,x82,x84,x87,x90,x92,x93,x94,x97,x98,x100,cat__Friday,cat__Monday,cat__Saturday,cat__Sunday,cat__Thursday,cat__Tuesday,cat__Wednesday,cat__April,cat__August,cat__December,cat__February,cat__January,cat__July,cat__June,cat__March,cat__May,cat__November,cat__October,cat__September
0,0.0,0.165254,18.060003,Wednesday,1.077380,-1.339233,-1.584341,0.0062,0.220784,1.816481,1.171788,4.644568,4.814885,-0.587361,4.278444,77.709700,-908.650758,7.328029,36.601967,126.384294,1.264713,3.834082,2.352406,905.491445,14.779467,0.752524,0,-0.597288,46.667221,3.159002,69.444841,32.450076,18.129499,1.227266,0.234654,53.359113,58.251700,1602.756464,2.740134,72.167186,-1.532562,300.629990,0,August,6.600008,1.252479,farmers,1.212048,12.896733,15.263491,0.518653,0.543768,-0.912506,53.521544,7.723719,-0.471594,-3.261865,0.383351,-0.162668,2.770844,0.909479,0,5.492487,7.627730,0,104.251338,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0
1,1.0,2.441471,18.416307,Friday,1.482586,0.920817,-0.759931,0.0064,1.192441,3.513950,1.419900,1.459868,1.443983,-1.165869,8.487132,70.808566,-1864.962288,6.796408,3.789176,93.037021,0.346201,4.345028,1.651579,868.784447,25.914680,1.445294,0,1.127376,47.928569,-1.013712,61.289132,35.521824,11.389081,-0.822055,1.104631,37.728834,56.052749,3350.837875,1.995635,73.964560,0.956001,300.629990,0,April,3.864254,0.057936,allstate,0.738526,14.402071,33.940951,-0.140384,1.016703,-0.000234,46.797571,7.260365,1.691428,-4.789183,2.138771,1.252022,3.703013,9.093466,0,3.346429,10.505284,1,101.230645,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
2,1.0,4.427278,19.188092,Thursday,0.145652,0.366093,0.709962,-0.0008,0.952323,0.782974,-1.247022,1.098525,1.216059,0.211685,8.601193,75.922820,-543.187403,3.802395,7.407996,71.022413,0.070968,4.332644,-0.375737,1077.233497,13.177479,4.174536,0,2.605279,56.674425,0.108486,69.798923,30.684074,14.372443,0.150728,-2.561083,50.236892,63.975108,3756.910196,1.772648,59.428690,4.127857,182.369349,0,September,1.370661,-0.239705,geico,-0.032980,14.402071,18.211817,-0.819798,1.010811,-0.000234,48.202036,9.336021,-2.608974,-6.387984,2.138771,0.657292,3.997183,0.305170,0,4.456565,7.810979,0,109.345215,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1
3,0.0,3.925235,19.901257,Tuesday,1.763602,-0.251926,-0.827461,-0.0057,-0.520756,1.825586,2.223038,-1.390239,3.962961,-2.046856,6.611554,74.966925,-182.626381,7.728963,8.136213,121.610846,0.700954,7.294990,-0.603983,1051.655489,17.006528,2.347355,0,1.071202,59.154933,1.319711,65.408246,34.401290,13.191173,-0.965711,-2.123048,41.857197,59.226119,1961.609788,3.155214,68.671023,0.663759,300.629990,0,September,0.013162,0.318335,geico,-0.466535,14.402071,14.629914,1.389325,0.704880,-1.510949,49.882647,5.661421,1.424316,-5.431998,2.138771,2.066346,2.933707,5.971782,0,4.100022,9.178325,1,103.021970,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1
4,0.0,2.868802,22.202473,Sunday,3.405119,0.083162,1.381504,0.0109,-0.732739,2.151990,-0.275406,7.230125,3.877312,-1.201565,9.103015,77.977156,967.007091,2.272765,24.452102,56.492317,-1.102387,6.313827,0.429187,949.904947,16.962710,0.510985,1,2.283921,46.445617,0.022747,66.662910,31.135261,17.210430,1.012093,3.477451,47.844153,55.921988,2345.195505,3.253079,71.853326,-2.273628,149.725023,0,January,1.305936,0.418272,geico,-0.195764,14.402071,12.227512,-2.951163,1.096548,-0.000234,51.349106,9.422401,0.705003,-5.664815,2.138771,0.663113,2.922302,1.046096,1,3.234033,11.702664,0,92.925935,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
5,0.0,1.926053,19.524376,Saturday,3.702574,2.300792,-0.331106,0.0079,3.085443,0.472701,1.943171,-3.582885,3.522031,-2.799700,7.283226,87.734440,271.163476,15.770180,44.874132,45.729121,0.320526,2.245217,4.303686,1187.333248,8.139008,2.233107,0,0.488196,45.559596,0.359074,76.835865,37.602918,12.628248,1.452704,-1.573853,52.359238,46.900080,2959.

#### x65

In [23]:
# insurance name
x65 = pd.get_dummies(dfCombine['x65'],prefix='cat_')
dfCombine = pd.merge(
    left=dfCombine,
    right=x65,
    left_index=True,
    right_index=True,
)

dfCombine.head(15)

,y,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x12,x13,x15,x17,x18,x19,x20,x21,x22,x23,x25,x26,x27,x28,x29,x31,x32,x34,x35,x36,x37,x40,x43,x46,x47,x48,x50,x51,x53,x56,x58,x59,x60,x62,x63,x65,x66,x67,x68,x69,x70,x71,x72,x73,x81,x82,x84,x87,x90,x92,x93,x94,x97,x98,x100,cat__Friday,cat__Monday,cat__Saturday,cat__Sunday,cat__Thursday,cat__Tuesday,cat__Wednesday,cat__April,cat__August,cat__December,cat__February,cat__January,cat__July,cat__June,cat__March,cat__May,cat__November,cat__October,cat__September,cat__allstate,cat__esurance,cat__farmers,cat__geico,cat__progressive
0,0.0,0.165254,18.060003,Wednesday,1.077380,-1.339233,-1.584341,0.0062,0.220784,1.816481,1.171788,4.644568,4.814885,-0.587361,4.278444,77.709700,-908.650758,7.328029,36.601967,126.384294,1.264713,3.834082,2.352406,905.491445,14.779467,0.752524,0,-0.597288,46.667221,3.159002,69.444841,32.450076,18.129499,1.227266,0.234654,53.359113,58.251700,1602.756464,2.740134,72.167186,-1.532562,300.629990,0,August,6.600008,1.252479,farmers,1.212048,12.896733,15.263491,0.518653,0.543768,-0.912506,53.521544,7.723719,-0.471594,-3.261865,0.383351,-0.162668,2.770844,0.909479,0,5.492487,7.627730,0,104.251338,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
1,1.0,2.441471,18.416307,Friday,1.482586,0.920817,-0.759931,0.0064,1.192441,3.513950,1.419900,1.459868,1.443983,-1.165869,8.487132,70.808566,-1864.962288,6.796408,3.789176,93.037021,0.346201,4.345028,1.651579,868.784447,25.914680,1.445294,0,1.127376,47.928569,-1.013712,61.289132,35.521824,11.389081,-0.822055,1.104631,37.728834,56.052749,3350.837875,1.995635,73.964560,0.956001,300.629990,0,April,3.864254,0.057936,allstate,0.738526,14.402071,33.940951,-0.140384,1.016703,-0.000234,46.797571,7.260365,1.691428,-4.789183,2.138771,1.252022,3.703013,9.093466,0,3.346429,10.505284,1,101.230645,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
2,1.0,4.427278,19.188092,Thursday,0.145652,0.366093,0.709962,-0.0008,0.952323,0.782974,-1.247022,1.098525,1.216059,0.211685,8.601193,75.922820,-543.187403,3.802395,7.407996,71.022413,0.070968,4.332644,-0.375737,1077.233497,13.177479,4.174536,0,2.605279,56.674425,0.108486,69.798923,30.684074,14.372443,0.150728,-2.561083,50.236892,63.975108,3756.910196,1.772648,59.428690,4.127857,182.369349,0,September,1.370661,-0.239705,geico,-0.032980,14.402071,18.211817,-0.819798,1.010811,-0.000234,48.202036,9.336021,-2.608974,-6.387984,2.138771,0.657292,3.997183,0.305170,0,4.456565,7.810979,0,109.345215,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0
3,0.0,3.925235,19.901257,Tuesday,1.763602,-0.251926,-0.827461,-0.0057,-0.520756,1.825586,2.223038,-1.390239,3.962961,-2.046856,6.611554,74.966925,-182.626381,7.728963,8.136213,121.610846,0.700954,7.294990,-0.603983,1051.655489,17.006528,2.347355,0,1.071202,59.154933,1.319711,65.408246,34.401290,13.191173,-0.965711,-2.123048,41.857197,59.226119,1961.609788,3.155214,68.671023,0.663759,300.629990,0,September,0.013162,0.318335,geico,-0.466535,14.402071,14.629914,1.389325,0.704880,-1.510949,49.882647,5.661421,1.424316,-5.431998,2.138771,2.066346,2.933707,5.971782,0,4.100022,9.178325,1,103.021970,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0
4,0.0,2.868802,22.202473,Sunday,3.405119,0.083162,1.381504,0.0109,-0.732739,2.151990,-0.275406,7.230125,3.877312,-1.201565,9.103015,77.977156,967.007091,2.272765,24.452102,56.492317,-1.102387,6.313827,0.429187,949.904947,16.962710,0.510985,1,2.283921,46.445617,0.022747,66.662910,31.135261,17.210430,1.012093,3.477451,47.844153,55.921988,2345.195505,3.253079,71.853326,-2.273628,149.725023,0,January,1.305936,0.418272,geico,-0.195764,14.402071,12.227512,-2.951163,1.096548,-0.000234,51.349106,9.422401,0.705003,-5.664815,2.138771,0.663113,2.922302,1.046096,1,3.234033,11.702664,0,92.925935,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0
5,0.0,1.926053,19.524376,Saturday,3.702574,2.300792,-0.331106,0.0079,3.085443,0.472701,1.943171,-3.582885,3.522031,-2.799700,7.283226,87.734440,271.163476,15.770180,44.874132,45.729121,0.320526,2.245217,4.303686,1187.333248,8.1

In [24]:
dfTrainClean = dfCombine.iloc[:40000,]
dfTestClean = dfCombine.iloc[40000:,]
# dfTrainClean.tail(1)
# print(len(dfTrainClean))


### Logistic regression (classification) & neural network

class sklearn.linear_model.LogisticRegression(penalty='l2',dual=False,tol=0.0001,C=1.0,fit_intercept=True,
intercept_scaling=1,class_weight=None,random_state=None,solver='lbfgs',max_iter=100,multi_class='auto', 
verbose=0,warm_start=False,n_jobs=None,l1_ratio=None)

In [26]:
dfCombine.columns

Index(['y', 'x1', 'x2', 'x3', 'x4', 'x5', 'x6', 'x7', 'x8', 'x9', 'x10', 'x12',
       'x13', 'x15', 'x17', 'x18', 'x19', 'x20', 'x21', 'x22', 'x23', 'x25',
       'x26', 'x27', 'x28', 'x29', 'x31', 'x32', 'x34', 'x35', 'x36', 'x37',
       'x40', 'x43', 'x46', 'x47', 'x48', 'x50', 'x51', 'x53', 'x56', 'x58',
       'x59', 'x60', 'x62', 'x63', 'x65', 'x66', 'x67', 'x68', 'x69', 'x70',
       'x71', 'x72', 'x73', 'x81', 'x82', 'x84', 'x87', 'x90', 'x92', 'x93',
       'x94', 'x97', 'x98', 'x100', 'cat__Friday', 'cat__Monday',
       'cat__Saturday', 'cat__Sunday', 'cat__Thursday', 'cat__Tuesday',
       'cat__Wednesday', 'cat__April', 'cat__August', 'cat__December',
       'cat__February', 'cat__January', 'cat__July', 'cat__June', 'cat__March',
       'cat__May', 'cat__November', 'cat__October', 'cat__September',
       'cat__allstate', 'cat__esurance', 'cat__farmers', 'cat__geico',
       'cat__progressive'],
      dtype='object')

In [27]:
features = [
    'x1','x2','x4','x5','x6','x7','x8','x9','x10','x12',
    'x13','x15','x17','x18', 'x19', 'x20', 'x21','x22', 'x23', 'x25',
    'x26','x27','x28','x29', 'x31', 'x32', 'x34','x35', 'x36', 'x37',
    'x40','x43','x46','x47', 'x48', 'x50', 'x51','x53', 'x56', 'x58',
    'x59','x62','x63','x66', 'x67', 'x68', 'x69','x70',
    'x71','x72','x73','x81', 'x82', 'x84', 'x87','x90','x92','x93',
    'x94','x97','x98', 'x100', 'cat__Friday', 'cat__Monday',
    'cat__Saturday','cat__Sunday','cat__Thursday', 'cat__Tuesday',
    'cat__Wednesday','cat__April','cat__August','cat__December',
    'cat__February','cat__January','cat__July','cat__June', 'cat__March',
    'cat__May','cat__November', 'cat__October', 'cat__September',
    'cat__allstate','cat__esurance', 'cat__farmers', 'cat__geico',
    'cat__progressive']

# x4 x6 x9 x10 x12 x17 x22 x23 x34 x48 x53 x59 x69 x73 x81 x92 x97
dropFeatures = [
    'x1','x2','x5','x7','x8',
    'x13','x15','x18','x19', 'x20','x21','x25',
    'x26','x27','x28','x29', 'x31','x32','x35', 'x36', 'x37',
    'x40','x43','x46','x47', 'x50','x51', 'x56','x58',
    'x62','x63','x66', 'x67', 'x68','x70',
    'x71','x72','x82','x84', 'x87','x90','x93',
    'x94','x98', 'x100', 'cat__Friday', 'cat__Monday',
    'cat__Saturday','cat__Sunday','cat__Thursday', 'cat__Tuesday',
    'cat__Wednesday','cat__April','cat__August','cat__December',
    'cat__February','cat__January','cat__July','cat__June', 'cat__March',
    'cat__May','cat__November', 'cat__October', 'cat__September',
    'cat__allstate','cat__esurance', 'cat__farmers', 'cat__geico',
    'cat__progressive']

### Scale data

In [28]:
scalerX = StandardScaler()
scalerX.fit(dfTrainClean[dropFeatures])
xTrainScaled = scalerX.transform(dfTrainClean[dropFeatures])
xTestScaled = scalerX.transform(dfTestClean[dropFeatures])

In [29]:
# logistic regression
logReg = LogisticRegression(C=7.7777,solver="lbfgs",max_iter=1111,tol=0.000003333, \
    multi_class="ovr",class_weight=None)

logReg.fit(xTrainScaled.astype(float),dfTrainClean["y"].astype(int))

LogisticRegression(C=7.7777, max_iter=1111, multi_class='ovr', tol=3.333e-06)

In [30]:
# neural networks
mlp = MLPClassifier(hidden_layer_sizes=(76,76,),random_state=None,alpha=0.1111, \
    solver='sgd',learning_rate="adaptive",activation="relu",n_iter_no_change=10, \
    tol=0.00001111,learning_rate_init=0.001111,max_iter=253,momentum=0.9111, \
    batch_size="auto",shuffle=True,warm_start=False,early_stopping=False,validation_fraction=0.1)

mlp.fit(xTrainScaled.astype(float),dfTrainClean["y"].astype(int))

C:\Users\TamLe\anaconda3\envs\sports\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (253) reached and the optimization hasn't converged yet.
  warnings.warn(


MLPClassifier(alpha=0.1111, hidden_layer_sizes=(76, 76),
              learning_rate='adaptive', learning_rate_init=0.001111,
              max_iter=253, momentum=0.9111, solver='sgd', tol=1.111e-05)

In [31]:
yPredTrainLog = logReg.predict(xTrainScaled.astype(float))
yPredTestLog = logReg.predict(xTestScaled.astype(float))
yProbTrainLog = logReg.predict_proba(xTrainScaled.astype(float))
yProbTestLog = logReg.predict_proba(xTestScaled.astype(float))

In [32]:
yPredTrainMLP = mlp.predict(xTrainScaled.astype(float))
yPredTestMLP = mlp.predict(xTestScaled.astype(float))
yProbTrainMLP = mlp.predict_proba(xTrainScaled.astype(float))
yProbTestMLP = mlp.predict_proba(xTestScaled.astype(float))

In [33]:
pd.DataFrame(yPredTrainLog).value_counts()

0    39238
1      762
dtype: int64

In [34]:
pd.DataFrame(dfTrain["y"]).value_counts()

y
0    34197
1     5803
dtype: int64

In [35]:
pd.DataFrame(yProbTrainLog).head(10)

,0,1
0,0.959971,0.040029
1,0.658801,0.341199
2,0.828818,0.171182
3,0.549976,0.450024
4,0.983968,0.016032
5,0.888512,0.111488
6,0.958273,0.041727
7,0.859887,0.140113
8,0.924804,0.075196
9,0.854813,0.145187


In [36]:
pd.DataFrame(yProbTestLog).head(15)

,0,1
0,0.979947,0.020053
1,0.972448,0.027552
2,0.740248,0.259752
3,0.822651,0.177349
4,0.986860,0.013140
5,0.942493,0.057507
6,0.725689,0.274311
7,0.940266,0.059734
8,0.673637,0.326363
9,0.376637,0.623363


In [37]:
pd.DataFrame(yProbTestLog).iloc[:,[1]].head(15)

,1
0,0.020053
1,0.027552
2,0.259752
3,0.177349
4,0.013140
5,0.057507
6,0.274311
7,0.059734
8,0.326363
9,0.623363


### Save results

In [39]:
pd.DataFrame(yProbTestLog).iloc[:,[1]].to_csv("glmresults.csv",header=False,index=False)
pd.DataFrame(yProbTestMLP).iloc[:,[1]].to_csv("nonglmresults.csv",header=False,index=False)

In [40]:
print(len(yProbTestMLP))

10000


### Statsmodels

In [41]:
xInclConstant = sm.add_constant(dfTrainClean[dropFeatures])
model = sm.Logit(dfTrainClean["y"].astype(int),xInclConstant.astype(float))
# logistic regression
results = model.fit()
results.summary()

Optimization terminated successfully.
         Current function value: 0.359918
         Iterations 7


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                      y   No. Observations:                40000
Model:                          Logit   Df Residuals:                    39933
Method:                           MLE   Df Model:                           66
Date:                Fri, 04 Jun 2021   Pseudo R-squ.:                  0.1308
Time:                        09:30:08   Log-Likelihood:                -14397.
converged:                       True   LL-Null:                       -16563.
Covariance Type:            nonrobust   LLR p-value:                     0.000
====================================================================================
                       coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------
const                3.5752   2.06e+06   1.73e-06      1.000   -4.04e+06    4.04e+06
x1                   0.0259      0.008      3.294      0.001       0.011       0.041
x2                   0.0954      0.022      4.407      0.000       0.053       0.138
x5                   0.0446      0.016      2.826      0.005       0.014       0.076
x7                 -74.1062      1.636    -45.290      0.000     -77.313     -70.899
x8                   0.1096      0.018      6.179      0.000       0.075       0.144
x13                  0.0194      0.009      2.108      0.035       0.001       0.037
x15                 -0.0205      0.014     -1.494      0.135      -0.047       0.006
x18                 -0.0094      0.005     -2.034      0.042      -0.018      -0.000
x19                 -0.0001   1.51e-05     -9.304      0.000      -0.000      -0.000
x20                 -0.0301      0.006     -4.903      0.000      -0.042      -0.018
x21                  0.0012      0.001      1.199      0.231      -0.001       0.003
x25                  0.0058      0.006      0.910      0.363      -0.007       0.018
x26                  0.0143      0.014      1.050      0.294      -0.012       0.041
x27                 -0.0004      0.000     -1.429      0.153      -0.001       0.000
x28                 -0.0973      0.012     -7.867      0.000      -0.122      -0.073
x29                 -0.0202      0.013     -1.574      0.115      -0.045       0.005
x31                 -0.5720      0.059     -9.740      0.000      -0.687      -0.457
x32                  0.0144      0.009      1.642      0.101      -0.003       0.032
x35                  0.0243      0.015      1.670      0.095      -0.004       0.053
x36                 -0.0115      0.010     -1.209      0.227      -0.030       0.007
x37                  0.0623      0.009      6.603      0.000       0.044       0.081
x40                 -0.1068      0.008    -13.138      0.000      -0.123      -0.091
x43                 -0.0168      0.015     -1.118      0.264      -0.046       0.013
x46                 -0.0549      0.012     -4.517      0.000      -0.079      -0.031
x47                 -0.1394      0.008    -16.839      0.000      -0.156      -0.123
x50              -9.406e-05   2.48e-05     -3.796      0.000      -0.000   -4.55e-05
x51                  0.0436      0.010      4.290      0.000       0.024       0.063
x56                 -0.0230      0.011     -2.158      0.031      -0.044      -0.002
x58                  0.0007      0.000      1.596      0.110      -0.000       0.002
x62                 -0.0533      0.013     -3.975      0.000      -0.080      -0.027
x63                  0.0444      0.013      3.485      0.000       0.019       0.069
x66                  0.0689      0.016      4.193      0.000       0.037       0.101
x67                 -0.0254      0.014     -1.867      0.062      -0.052       0.001
x68                  0.0145      0.003      4.265      0.000       0.008       0.021
x70        

In [42]:
# categorical_columns = ['Gender', 'House Type']
# for column in categorical_columns:
#     tempdf = pd.get_dummies(df[column], prefix=column)
#     df = pd.merge(
#         left=df,
#         right=tempdf,
#         left_index=True,
#         right_index=True,
#     )
#     df = df.drop(columns=column)
# print(df)

### AUC curve on training set
There is no dependent variable in test set.

In [43]:
yTrainClean = dfTrainClean["y"].to_numpy()
print(yTrainClean)
falsePositiveRate,truePositiveRate,threshold = metrics.roc_curve(yTrainClean,yProbTrainLog[:,1])

rocAucScore = metrics.roc_auc_score(yTrainClean,yProbTrainLog[:,1])
print("auc score: %s" % rocAucScore)
logLoss = log_loss(yTrainClean,yProbTrainLog[:,1])
print("log loss: %s" % logLoss)

[0. 1. 1. ... 0. 0. 0.]
auc score: 0.7527958790394674
log loss: 0.35991762698348395


In [44]:
yTrainClean = dfTrainClean["y"].to_numpy()
print(yTrainClean)
falsePositiveRate,truePositiveRate,threshold = metrics.roc_curve(yTrainClean,yProbTrainMLP[:,1])

rocAucScore = metrics.roc_auc_score(yTrainClean,yProbTrainMLP[:,1])
print("auc score: %s" % rocAucScore)
logLoss = log_loss(yTrainClean,yProbTrainMLP[:,1])
print("log loss: %s" % logLoss)

[0. 1. 1. ... 0. 0. 0.]
auc score: 0.8287156426985424
log loss: 0.31830866329503515
